## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop('EIN', axis = 1)
application_df = application_df.drop('NAME' , axis = 1)
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
unique_columns = application_df.nunique()
unique_columns

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_unique = application_df['APPLICATION_TYPE'].value_counts()
app_unique

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      260
T2          16
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_unique = application_df['CLASSIFICATION'].value_counts()
class_unique

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2380        1
C2570        1
C1900        1
C1570        1
C8210        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_unique_nonzero = class_unique[class_unique > 1]
class_unique_small = class_unique[class_unique < 1883]
df_class = pd.DataFrame(class_unique_small)
classifications_to_replace = df_class.index
classifications_to_replace

Index(['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100',
       'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000',
       'C7120', 'C1500', 'C6000', 'C1800', 'C1250', 'C8200', 'C1278', 'C1238',
       'C1235', 'C1237', 'C7210', 'C4100', 'C1720', 'C2400', 'C1257', 'C1600',
       'C1260', 'C0', 'C2710', 'C1267', 'C1246', 'C1256', 'C1234', 'C3200',
       'C4200', 'C1370', 'C2500', 'C5200', 'C1820', 'C1732', 'C1580', 'C1236',
       'C1245', 'C2150', 'C3700', 'C1248', 'C2170', 'C4500', 'C6100', 'C2190',
       'C2561', 'C2600', 'C4120', 'C1283', 'C1728', 'C2380', 'C2570', 'C1900',
       'C1570', 'C8210'],
      dtype='object')

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
with_dummies = pd.get_dummies(application_df)
with_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0


In [16]:
# Split our preprocessed data into our features and target arrays
y = with_dummies['IS_SUCCESSFUL']
X = with_dummies.drop('IS_SUCCESSFUL', axis = 1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [17]:
#len(x_train[0])
X_train

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
10679,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5052,1,5000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
9990,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
25173,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11405,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7813,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
32511,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
5192,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
12172,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
# Create a StandardScaler instances
from sklearn.model_selection import train_test_split
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'relu', input_dim = 44))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))


# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1)                 45        
                                                                 
 dense_4 (Dense)             (None, 1)                 2         
                                                                 
 dense_5 (Dense)             (None, 1)                 2         
                                                                 
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 1s 626us/step - loss: 0.6914 - accuracy: 0.5327
Epoch 2/100
804/804 [==============================] - 1s 634us/step - loss: 0.6910 - accuracy: 0.5335
Epoch 3/100
804/804 [==============================] - 1s 640us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 4/100
804/804 [==============================] - 1s 629us/step - loss: 0.6910 - accuracy: 0.5335
Epoch 5/100
804/804 [==============================] - 1s 624us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 6/100
804/804 [==============================] - 0s 615us/step - loss: 0.6910 - accuracy: 0.5335
Epoch 7/100
804/804 [==============================] - 0s 617us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 8/100
804/804 [==============================] - 0s 607us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 9/100
804/804 [==============================] - 0s 584us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 10/100
804/804 [==============================] - 0s 580us/step - l

804/804 [==============================] - 1s 795us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 80/100
804/804 [==============================] - 1s 749us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 81/100
804/804 [==============================] - 1s 819us/step - loss: 0.6910 - accuracy: 0.5335
Epoch 82/100
804/804 [==============================] - 1s 804us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 83/100
804/804 [==============================] - 1s 829us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 84/100
804/804 [==============================] - 1s 845us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 85/100
804/804 [==============================] - 1s 794us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 86/100
804/804 [==============================] - 1s 813us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 87/100
804/804 [==============================] - 1s 799us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 88/100
804/804 [==============================] - 1s 799us/step - loss:

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy:{model_accuracy}")

268/268 - 0s - loss: 0.6915 - accuracy: 0.5292 - 175ms/epoch - 653us/step
Loss: 0.6914675831794739, Accuracy:0.5292128324508667


In [24]:
# Export our model to HDF5 file
import numpy as np
import h5py


In [30]:
nn.get_weights()

[array([[ 7.15940893e-02],
        [-2.58951247e-01],
        [ 1.82838380e-01],
        [ 5.45551777e-02],
        [-2.41973698e-01],
        [ 2.94870138e-01],
        [ 2.59662867e-01],
        [ 3.51377845e-01],
        [ 4.22239304e-04],
        [ 1.50005281e-01],
        [-2.40518436e-01],
        [ 1.42979026e-02],
        [-1.10575944e-01],
        [-2.53646910e-01],
        [-7.28125572e-02],
        [-1.19497508e-01],
        [ 3.25496852e-01],
        [-2.01927736e-01],
        [-2.61787623e-01],
        [-3.24364483e-01],
        [-1.06104821e-01],
        [ 4.15650308e-02],
        [-2.57029444e-01],
        [-3.07367265e-01],
        [-9.61802006e-02],
        [-2.43984833e-01],
        [-1.65620223e-01],
        [ 1.78012609e-01],
        [-1.88869178e-01],
        [ 2.94417143e-04],
        [ 1.36849999e-01],
        [ 3.61089885e-01],
        [ 2.83581376e-01],
        [-2.68991172e-01],
        [-2.62608737e-01],
        [ 2.32610226e-01],
        [ 2.27544308e-01],
 

In [40]:
model_json = nn.to_json()
with open('model.json','w',encoding='UTF-8') as json_file:
    json_file.write(model_json)
nn.save_weights('Nataliem.h5',save_format='h5',)
print('saved model to disk')

saved model to disk
